In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

import bmll2 as b2
from bmll2 import reference, Security, NormalisedSecurity, SparkHelper, get_market_data, get_market_data_range, VenueMarketError, save_spark_dataframe, load_spark_dataframe

pd.options.display.max_columns = None

In [43]:
#jse_securities = reference.query(MIC = 'XJSE')
#jse_securities = jse_securities[(jse_securities['InstrumentType'] == 'Equity') & (jse_securities['IsAlive'] == True)]
#jse_listing_ids = dict(zip(jse_securities['Ticker'], jse_securities['ListingId']))
#jse_tickers    = jse_securities['Ticker']

In [44]:
#jse_listing_ids['AGL']
#jse_securities[jse_securities['Ticker'] == 'AGL'][['Ticker', 'ListingId']]
#jse_securities.shape[0]

769359338

In [ ]:
#trades = market_data.all_trades()
#trades = trades[trades['market_state'] == 'CONTINUOUS_TRADING']
#trades.aggressor_side.value_counts()
#trades.head()

In [ ]:
#incremental_book = market_data.incremental_book_L3()
#incremental_book = incremental_book[incremental_book['market_state'] == 'CONTINUOUS_TRADING']
#incremental_book[incremental_book['order_executed'] == True]

In [ ]:
#sec = NormalisedSecurity.from_listing_id(769359338, '2025-12-12')
#md = sec.market_data()
#bbo = md.best_bid_offer()
#bbo = bbo[bbo.market_state == 'CONTINUOUS_TRADING'].sort_values('event_timestamp')
#bbo.head()
#mid_price = (bbo.best_bid_price + bbo.best_ask_price) / 2
#vol = np.log(mid_price.max()) - np.log(mid_price.min())
#vol

In [ ]:
#%%time
#test = get_market_data_range('XJSE', start_date = '2023-01-01', end_date = '2023-02-28', ticker = 'AGL',
#                             table_name = 'l1', df_engine = 'polars', columns = ['TradeDate', 'BidPrice1', 'AskPrice1', 'MarketState'])

In [ ]:
#test = test.to_pandas()
#test = test[test['MarketState'] == 'CONTINUOUS_TRADING']
#test.MarketState.unique()
#midprice = (test['AskPrice1'] + test['BidPrice1']) / 2
#test.insert(loc = 1, column = 'Mid-price', value = midprice)

#daily_vol = (test.groupby('TradeDate')['Mid-price'].apply(lambda x: np.log(x.max()) - np.log(x.min()))).rename('Daily Volatility')
#trade_plus.insert(loc = 5, column = 'Daily Volatility', value = daily_vol)
#trade_plus.insert(loc = 6, column = 'Daily Volume', value = trade_plus['Volume'].sum())
#trade_plus.head()
#test.head()
#daily_vol
#test['Daily Volatility'] = test['TradeDate'].map(daily_vol)
#test.head()
#test = pd.merge(test, daily_vol, on = "TradeDate", how = "left")
#test.sort_values('EventTimestamp').head()

In [3]:
#%%time
trade_plus = get_market_data_range('XJSE', start_date = '2023-01-01', end_date = '2023-06-30',
                                   table_name = 'trades-plus', df_engine = 'polars', ticker = 'ABG',
                                   columns = ['Classification', 'MIC', 'Ticker', 'ListingId', 'TradeDate',
                                              'LocalTradeTimestamp', 'ExchangeSequenceNo', 'AggressorSide',
                                              'Price', 'Size', 'PreTradeMid1ms', 'PostTradeMid1ms'])
trade_plus = trade_plus.to_pandas()
trades_plus = trade_plus.groupby('TradeDate')
trade_plus = trade_plus[(trade_plus['Classification'] == 'LIT_CONTINUOUS')]
trade_plus = trade_plus[['MIC', 'Ticker', 'ListingId', 'TradeDate', 'LocalTradeTimestamp', 'ExchangeSequenceNo',
                         'AggressorSide', 'Price', 'Size', 'PreTradeMid1ms', 'PostTradeMid1ms']]
trade_plus['AggressorSide'] = trade_plus['AggressorSide'].map({1 : 1, 2 : -1, 0 : 0})
trade_plus = trade_plus.rename(columns = {'AggressorSide' : 'Trade Sign', 'PreTradeMid1ms' : 'Mid-price before',
                                          'PostTradeMid1ms' : 'Mid-price after(immediate)', 'Size' : 'Volume',
                                          'LocalTradeTimestamp' : 'DateTime', 'TradeDate' : 'Date'})
midprice_after_delayed = trade_plus['Mid-price before'].shift(-1)
trade_plus.insert(loc = 10, column = 'Mid-price after(delayed)', value = midprice_after_delayed)
trade_plus.sort_values('DateTime').shape[0]

957642

In [ ]:
#l1 = get_market_data_range('XJSE', start_date = '2023-01-10', end_date = '2023-01-10', ticker = 'RMH',
#                            table_name = 'l1', df_engine = 'polars')
#l1 = l1.to_pandas()
#l1 = l1[l1['MarketState'] == 'CONTINUOUS_TRADING']
#midprice = (l1['AskPrice1'] + l1['BidPrice1']) / 2
#l1.insert(loc = 5, column = 'Mid-price', value = midprice)
#l1 = l1.rename(columns = {'TradeDate' : 'Date'})
#daily_vol = (l1.groupby('Date')['Mid-price'].apply(lambda x: np.log(x.max()) - np.log(x.min()))).rename('Daily Volatility')
#daily_vol
#trade_plus = pd.merge(trade_plus, daily_vol, on = 'Date', how = 'left')
#trade_plus.insert(loc = 6, column = 'Daily Volume', value = trade_plus['Volume'].sum())
#trade_plus.sort_values(['DateTime', 'ExchangeSequenceNo'])
#trade_plus.head()

In [35]:
#midprice

In [ ]:
#trade_plus[(trade_plus['DateTime'] >= '2023-01-03 16:45:58.046172') & 
#            (trade_plus['DateTime'] <= '2023-01-04 09:01:58.046172')]

In [ ]:
#l1 = get_market_data_range('XJSE', start_date = '2023-01-05', end_date = '2023-01-05', ticker = 'AGL',
#                            table_name = 'l1', df_engine = 'polars')
#l1 = l1.to_pandas()
#l1 = l1[l1['MarketState'] == 'CONTINUOUS_TRADING']
#midprice = (l1['AskPrice1'] + l1['BidPrice1']) / 2
#l1.insert(loc = 5, column = 'Mid-price', value = midprice)
#l1 = l1.rename(columns = {'TradeDate' : 'Date'})
#daily_vol = (l1.groupby('Date')['Mid-price'].apply(lambda x: np.log(x.max()) - np.log(x.min()))).rename('Daily Volatility')
#trade_plus = pd.merge(trade_plus, daily_vol, on = 'Date', how = 'left')
#trade_plus.insert(loc = 5, column = 'Daily Volatility', value = daily_vol)
#trade_plus.insert(loc = 6, column = 'Daily Volume', value = trade_plus['Volume'].sum())
#trade_plus.sort_values(['DateTime', 'ExchangeSequenceNo'])
#daily_vol

In [13]:
def get_data(ticker, starting_date, ending_date):

    #if isinstance(startingdate, str):
    #    date = pd.to_datetime(date).date()
    
    try:
        trade_plus = get_market_data_range('XJSE',start_date = starting_date, end_date = ending_date,
                                   table_name = 'trades-plus', df_engine = 'polars', ticker = ticker,
                                   columns = ['Classification', 'MIC', 'Ticker', 'ListingId', 'TradeDate',
                                              'LocalTradeTimestamp', 'ExchangeSequenceNo', 'AggressorSide',
                                              'Price', 'Size', 'PreTradeMid1ms', 'PostTradeMid1ms'])
    except VenueMarketError:
        return None
        
    trade_plus = trade_plus.to_pandas()
    trade_plus = trade_plus[(trade_plus['Classification'] == 'LIT_CONTINUOUS')]
    
    if trade_plus is None or trade_plus.empty:
        return None
        
    trade_plus = trade_plus[['MIC', 'Ticker', 'ListingId', 'TradeDate', 'LocalTradeTimestamp', 'ExchangeSequenceNo',
                             'AggressorSide', 'Price', 'Size', 'PreTradeMid1ms', 'PostTradeMid1ms']]
    trade_plus['AggressorSide'] = trade_plus['AggressorSide'].map({1 : 1, 2 : -1, 0 : 0})
    trade_plus = trade_plus.rename(columns = {'AggressorSide' : 'Trade Sign', 'PreTradeMid1ms' : 'Mid-price before',
                                              'PostTradeMid1ms' : 'Mid-price after(immediate)', 'Size' : 'Volume',
                                              'LocalTradeTimestamp' : 'DateTime', 'TradeDate' : 'Date'})
    midprice_after_delayed = trade_plus['Mid-price before'].shift(-1)
    trade_plus.insert(loc = 11, column = 'Mid-price after(delayed)', value = midprice_after_delayed)
    
    l1 = get_market_data_range('XJSE', start_date = starting_date, end_date = ending_date, ticker = ticker,
                               table_name = 'l1', df_engine = 'polars')
    l1 = l1.to_pandas()
    l1 = l1[l1['MarketState'] == 'CONTINUOUS_TRADING']
    midprice = (l1['AskPrice1'] + l1['BidPrice1']) / 2
    l1.insert(loc = 5, column = 'Mid-price', value = midprice)
    l1 = l1.rename(columns = {'TradeDate' : 'Date'})
    daily_vol = (l1.groupby('Date', as_index = False)['Mid-price'].apply(lambda x: np.log(x.max()) - np.log(x.min()))).rename(columns = {'Mid-price' : 'Daily Volatility'})
    trade_plus = pd.merge(trade_plus, daily_vol, on = 'Date', how = 'left')

    col = 'Daily Volatility'
    cols = list(trade_plus.columns)
    cols.insert(6, cols.pop(cols.index(col)))
    trade_plus = trade_plus[cols]
    daily_volume = (trade_plus.groupby('Date', as_index = False)['Volume'].sum().rename(columns = {'Volume' : 'Daily Volume'}))
    trade_plus = pd.merge(trade_plus, daily_volume, on = 'Date', how = 'left')
    col = 'Daily Volume'
    cols = list(trade_plus.columns)
    cols.insert(6, cols.pop(cols.index(col)))
    trade_plus = trade_plus[cols]
    #trade_plus.insert(loc = 6, column = 'Daily Volume', value = trade_plus['Volume'].sum())
    
    trade_plus.sort_values(['DateTime', 'ExchangeSequenceNo'])
    return trade_plus
    

In [14]:
top40_tickers = [
    "ABG",  # ABSA
    "AGL",  # Anglo American
    "ANG",  # Anglogold Ashanti
    "ANH",  # AB InBev
    "APN",  # Aspen Pharmacare
    "BHG",  # BHP Group
    "BID",  # Bidcorp
    "BVT",  # Bidvest
    "BTI",  # British American Tobacco
    "CPI",  # Capitec
    "CLS",  # Clicks
    "DSY",  # Discovery  
    "EXX",  # Exxaro
    "FSR",  # FirstRand
    "GLN",  # Glencore
    "GFI",  # Gold Fields
    "GRT",  # Growthpoint
    "IMP",  # Impala Platinum
    "INL",  # Investec Ltd
    "INP",  # Investec PLC
    "MNP",  # Mondi
    "MRP",  # Mr Price
    "MTN",  # MTN Group
    "NPN",  # Naspers
    "NED",  # Nedbank
    "NRP",  # NEPI Rockcastle
    "OMU",  # Old Mutual
    "PRX",  # Prosus
    "RNI",  # Reinet Investments
    "REM",  # Remgro
    "RMH",  # RMB
    "SLM",  # Sanlam
    "SOL",  # Sasol
    "SHP",  # Shoprite
    "SBK",  # Standard Bank
    "VAL",  # Valterra platinum
    "VOD",  # Vodacom
    "WHL"   # Woolworths
]

start_dates = pd.date_range(start = '2023-01-01', end = '2025-12-31', freq = 'MS', inclusive = 'both').date
end_dates   = pd.date_range(start = '2023-01-01', end = '2025-12-31', freq = 'M', inclusive = 'both').date

In [15]:
for ticker in top40_tickers:
    print(ticker)
    stock = []
    for i in range(len(start_dates)):
        data = get_data(ticker, start_dates[i], end_dates[i])
        if data is not None:
            stock.append(data)
    
    stock_data = pd.concat(stock, ignore_index = True)
    stock_data.to_csv(f'{ticker}.csv', index = False)
    b2.put_file(f'{ticker}.csv', 'top_40')

ABG
AGL
ANG
ANH
APN
BHG
BID
BVT
BTI
CPI
CLS
DSY
EXX
FSR
GLN
GFI
GRT
IMP
INL
INP
MNP
MRP
MTN
NPN
NED
NRP
OMU
PRX
RNI
REM
RMH
SLM
SOL
SHP
SBK
VAL
VOD
WHL


In [ ]:
# I need to do this because the ticker for VAL was AMS before 2025-05-28
%%time
start_dates = pd.date_range(start = '2023-01-01', end = '2025-04-30', freq = 'MS', inclusive = 'both').date
end_dates   = pd.date_range(start = '2023-01-01', end = '2025-04-30', freq = 'M', inclusive = 'both').date

VAL = []
for i in range(len(start_dates)):
    print(start_dates[i], end_dates[i])
    data = get_data('AMS', start_dates[i], end_dates[i])
    if data is not None:
        VAL.append(data)

start_dates = pd.date_range(start = '2025-05-01', end = '2025-05-27', freq = 'B', inclusive = 'both').date
end_dates   = pd.date_range(start = '2025-05-01', end = '2025-05-27', freq = 'B', inclusive = 'both').date

for i in range(len(start_dates)):
    print(start_dates[i], end_dates[i])
    data = get_data('AMS', start_dates[i], end_dates[i])
    if data is not None:
        VAL.append(data)

start_dates = pd.date_range(start = '2025-05-27', end = '2025-05-31', freq = 'B', inclusive = 'both').date
end_dates   = pd.date_range(start = '2025-05-27', end = '2025-05-31', freq = 'B', inclusive = 'both').date

for i in range(len(start_dates)):
    print(start_dates[i], end_dates[i])
    data = get_data('VAL', start_dates[i], end_dates[i])
    if data is not None:
        VAL.append(data)

start_dates = pd.date_range(start = '2025-06-01', end = '2025-12-31', freq = 'MS', inclusive = 'both').date
end_dates   = pd.date_range(start = '2025-06-01', end = '2025-12-31', freq = 'M', inclusive = 'both').date

for i in range(len(start_dates)):
    print(start_dates[i], end_dates[i])
    data = get_data('VAL', start_dates[i], end_dates[i])
    if data is not None:
        VAL.append(data)

VAL_data = pd.concat(VAL, ignore_index = True)
VAL_data['Ticker'] = 'VAL'
VAL_data.to_csv('VAL.csv', index = False)
b2.put_file('VAL.csv', 'top_40')